In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [13]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

## This is how to make a custom tool

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
db=FAISS.from_documents(documents,OpenAIEmbeddings())

retriever=db.as_retriever()





[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content="Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricin

In [12]:
# create retriever
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                                            "Search for information on LangSmith.For any question about langsmith you must use this tool")

retriever_tool.name

'langsmith_search'

In [15]:
# Arxiv tool
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper)
arxiv.name

'arxiv'

### Combine all the tools

In [16]:
tools=[wiki,arxiv,retriever_tool]

In [18]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

llm = OpenAI()

load_dotenv()
key=os.getenv("OPENAI_API_KEY")

llm=OpenAI(api_key=key, model="gpt-3.5-turbo")

In [20]:
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\KHIZAR\AppData\Local\pypoetry\Cache\virtualenvs\langchain-dQg_le0Q-py3.12\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [21]:
from langchain.agents import create_openai_tools_agent

agent=create_openai_tools_agent(llm,tools,prompt)

In [23]:
# agent executer
from langchain.agents import AgentExecutor

agent_executor=AgentExecutor.from_agent_and_tools(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [25]:
agent_executor.invoke({"input":"Tell me abouy deep learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Deep learning'}`


Page: Deep learning
Summary: Deep learning is the subset of machine learning methods based on neural networks with representation learning. The adjective "deep" refers to the use of multiple layers inDeep learning is a subset of machine learning methods that are based on neural networks with representation learning. The term "deep" in deep learning refers to the use of multiple layers in the neural network architecture. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of complex models that can learn from large amounts of data. If you would like more detailed information, feel free to ask!

> Finished chain.


{'input': 'Tell me abouy deep learning',
 'output': 'Deep learning is a subset of machine learning methods that are based on neural networks with representation learning. The term "deep" in deep learning refers to the use of multiple layers in the neural network architecture. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of complex models that can learn from large amounts of data. If you would like more detailed information, feel free to ask!'}